## Computing and plotting wind speed (GRIB)

In this notebook we will demonstrate how to:

* extract wind components on the 500 hPa pressure level
* compute the wind speed
* change the GRIB header in the resulting fields
* plot the results

In [1]:
import metview as mv

We are going to use wind forecast data from MARS for a set of pressure levels and forecast steps on a 5x5 degree regular latlon grid. If you have retrieved the data earlier, Metview can read it straight away. Alternatively, Metview can also retrieve it from MARS.

In [2]:
filename = "uv.grib"
if mv.exist(filename):
    g = mv.read(filename)
else:
    g = mv.retrieve(type='fc', 
                    date = -1, 
                    time = 0,
                    param = ['u','v'], 
                    levelist = ['500', '1000'], 
                    levType = 'pl',
                    step = [0, 6, 12, 18],            
                    grid = [5, 5])
    mv.write(filename, g)

The result is a [Fieldset](https://metview.readthedocs.io/en/latest/data_types/fieldset.html), which is Metview's own class to represent GRIB data. We can check the contents of our fieldset with [describe()](https://metview.readthedocs.io/en/latest/api/functions/describe.html), which shows us high level content of the grib.

In [3]:
g.describe()

parameter,typeOfLevel,level,date,time,step,paramId,class,stream,type,experimentVersionNumber
u,isobaricInhPa,"500,1000",20240418,0,"0,6,...",131,od,oper,fc,0001
v,isobaricInhPa,"500,1000",20240418,0,"0,6,...",132,od,oper,fc,0001


Next, we **filter** the 500 hPa wind components using the **[select()](https://metview.readthedocs.io/en/latest/api/functions/select.html)** function.

In [4]:
u = g.select(shortName='u', level=500)
v = g.select(shortName='v', level=500)

Having extracted the wind components we **compute** the wind speed fields using Metview's high level fieldset arithmetic. 

In [5]:
sp = mv.sqrt(u*u + v*v)

We can see that in the results the **paramId** and the corresponding **shortName** still refer to the u wind component.

In [6]:
mv.grib_get(sp, ['shortName', 'paramId','level', 'step'])

[['u', '131', '500', '0'],
 ['u', '131', '500', '6'],
 ['u', '131', '500', '12'],
 ['u', '131', '500', '18']]

The reason for this is that Metview always copies the metadata of the first field in the expression into the resulting field. When we plot fields, the automatic contouring and title are based on the **paramId**. So it is generally a good idea to set it correctly. We will do it by using the [grib_set_long()](../api/functions/grib_set.rst#grib_set_long) function (changing the paramId will set the shortName automatically).

In [7]:
sp = mv.grib_set_long(sp, ['paramId', 10])
mv.grib_get(sp, ['shortName', 'paramId','level', 'step'])

[['ws', '10', '500', '0'],
 ['ws', '10', '500', '6'],
 ['ws', '10', '500', '12'],
 ['ws', '10', '500', '18']]

Finally, we **plot** the first windspeed field using automatic contouring and title settings.

In [8]:
mv.setoutput('jupyter')
mv.plot(sp, mv.mcont(contour_automatic_setting='ecmwf', legend='on'))

Image(value=b'', layout="Layout(visibility='hidden')")

Label(value='Generating plots....')

This notebook is adapted from [the example in Metview documentation](https://metview.readthedocs.io/en/latest/examples/compute_wind_speed.html).